In [ ]:
import pandas as pd 
import numpy as np 

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Flatten


import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_dir='/kaggle/input/dogs-vs-cats/train'
val_dir='/kaggle/input/dogs-vs-cats/test'

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,        # Path to the directory containing image data.
    labels="inferred",          # Automatically infer labels from subdirectory names.
    label_mode='int',   # Labels are returned as one-hot encoded arrays (use "categorical" for classification tasks).
    class_names=None,           # Class names inferred from directory structure if None; can be manually specified.
    color_mode="rgb",           # Load images in RGB color mode (use "grayscale" for grayscale images).
    batch_size=32,              # Number of samples per batch of data.
    image_size=(256, 256),      # Resize all images to this target size (256x256 pixels in this case).
    validation_split=0.2,       # Reserve 20% of the data for validation.
    subset="training",          # Specify that this dataset is for training. Use "validation" for validation data.
    seed=123                   # Seed for shuffling and ensuring consistent split between training and validation.
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=val_dir,
    labels="inferred",
    label_mode='int',  # Use "categorical" for categorical labels
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    validation_split=0.2,
    subset="validation",
    seed=123
)


In [ ]:
for image, label in train_ds.take(1):  # Take one batch from the dataset
    print("Image shape:", image.shape)  # Print the shape of the image tensor
    print("Label shape:", label.shape)  # Print the shape of the label tensor


In [ ]:
import matplotlib.pyplot as plt

# Get a batch of images and labels
images, labels = next(iter(train_ds.take(1)))#etrieves the first batch of images and labels

# Display the first 9 images
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_ds.class_names[labels[i]])
    plt.axis("off")
plt.show()


In [ ]:
def processes(image,label):
    image=tf.cast(image/255. ,tf.float32)# 256 -->0 to 255 and cast to float
    return image,label

train_ds=train_ds.map(processes)
val_ds=val_ds.map(processes)

In [ ]:
model=Sequential()
#1st convolutional  layer
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
#2st convolutional  layer
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
#2nd convolutional  layer
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

#flattern convert to 1D
model.add(Flatten())

#add fully connected neural network
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))


In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
#Early_Stopping
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=5, 
    restore_best_weights=True,
)
#fit model
history=model.fit(train_ds,epochs=10,validation_data=val_ds,callbacks = early_stopping)

In [ ]:
validation_loss, validation_accuracy = model.evaluate(val_ds)

# Print the validation loss and accuracy
print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], color='red', label='Train Accuracy')
plt.plot(history.history['val_accuracy'], color='blue', label='Validation Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.show()


In [ ]:
#way to reduce overfitting
#add more data
#data augmentation
#l1/l2 regularization
#dropout
#batchnormalization
#pretrained model

In [ ]:
import cv2
test_img=cv2.imread('/kaggle/input/dogs-vs-cats/test/dogs/dog.10028.jpg')
plt.imshow(test_img)
test_img=cv2.resize(test_img,(256,256))
test_input=test_img.reshape(1,256,256,3)
model.predict(test_input)

In [ ]:
from keras import layers, models, optimizers
from keras.applications import VGG16


conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(256, 256, 3))

model2 =Sequential()
model2.add(conv_base)
model2.add(layers.Flatten())
model2.add(layers.Dropout(0.4))
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

conv_base.trainable = False

model2.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=2e-5),   #2*10^-5=0.00002
              metrics=['acc'])

In [ ]:
history=model2.fit(train_ds,epochs=10,validation_data=val_ds,callbacks = early_stopping)

In [ ]:
validation_loss, validation_accuracy = model2.evaluate(val_ds)

# Print the validation loss and accuracy
print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['acc'], color='red', label='Train Accuracy')
plt.plot(history.history['val_acc'], color='blue', label='Validation Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.show()


In [ ]:
from keras.applications import Xception

conv_base = Xception(weights='imagenet',
                  include_top=False,
                  input_shape=(256, 256, 3))

model3 =Sequential()
model3.add(conv_base)
model3.add(layers.Flatten())
model3.add(layers.Dropout(0.4))
model3.add(layers.Dense(256, activation='relu'))
model3.add(layers.Dense(1, activation='sigmoid'))

conv_base.trainable = False

model3.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=2e-5),#2*10^-5=0.00002
              metrics=['acc'])

In [ ]:
history=model3.fit(train_ds,epochs=10,validation_data=val_ds,callbacks = early_stopping)

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['acc'], color='red', label='Train Accuracy')
plt.plot(history.history['val_acc'], color='blue', label='Validation Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.show()


In [ ]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(256, 256, 3))

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False


model4 = models.Sequential()
model4.add(conv_base)
model4.add(layers.Flatten())
model4.add(layers.Dropout(0.5))
model4.add(layers.Dense(256, activation='relu'))
model4.add(layers.Dense(1, activation='sigmoid'))

model4.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-5),#2*10^-5=0.00002
              metrics=['acc'])

In [ ]:
history=model4.fit(train_ds,epochs=10,validation_data=val_ds,callbacks = early_stopping)